### Tokenizing + Vectorization

In [10]:
import string
import pandas as pd # pip install pandas
import nltk # pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer # pip install scikit-learn
from sklearn import decomposition

In [7]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

df = pd.read_csv("../dataset/_compiled/Compiled.csv")

In [11]:
def remove_punctuation(cell):
    return cell.translate(str.maketrans('', '', string.punctuation))

def tokenizer(text):
    word_tokens = [x for x in word_tokenize(text) if len(x) > 3]
    filtered_tokens = [word for word in word_tokens if word not in set([remove_punctuation(x) for x in [*stopwords.words('english'), "im", "were", "youre", "thats", "theres"]])]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return lemmatized_tokens

In [12]:
tf_vectorizer = TfidfVectorizer(tokenizer=tokenizer,
                                max_df=0.75, max_features=10000, 
                                use_idf=True, norm=None, token_pattern=None)
tf_vectors = tf_vectorizer.fit_transform(df["Translated"])  